In [ ]:
import requests

OSRM_URL = "http://localhost:5000"

import pandas as pd
df = pd.read_csv('train_cleaned.csv', index_col=0)


def get_osrm_route(start_lat, start_lng, end_lat, end_lng):
    """Fetch route details from OSRM based on start and end coordinates."""
    coordinates = f"{start_lng},{start_lat};{end_lng},{end_lat}"
    route_url = f"{OSRM_URL}/route/v1/driving/{coordinates}?overview=false"  # Change 'overview' if full route is needed

    response = requests.get(route_url)
    
    if response.status_code == 200:
        data = response.json()
        if data["routes"]:  # Ensure routes are returned
            return data["routes"][0]["distance"], data["routes"][0]["duration"]
    return None, None  # Return None if no valid response

df["osrm_distance"], df["osrm_duration"] = zip(*df.apply(lambda row: get_osrm_route(
    row["start_lat"], row["start_lng"], row["end_lat"], row["end_lng"]), axis=1))


df.to_csv('train_with_osrm.csv', index=False)